In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [32]:
# os.environ['OPENAI_API_KEY'] = "OPENAI_API_KEY"
# os.environ['SERPAPI_API_KEY'] = "SERPAPI_API_KEY"

In [17]:
from langchain.llms import OpenAI
llm = OpenAI(temperature=0.6)
name = llm("I want to open a restaurant for Italian food. Suggest a fency name for this.")
print(name)



"Bella Cucina" 


### I don't want to keep on changing the prompt. So we create Prompt Tempaltes

In [11]:
from langchain.prompts import PromptTemplate
prompt_template_name = PromptTemplate(
    input_variables=['cuisine'],
    template= "I want to open a restaurant for {cuisine} food. Suggest a fency name for this.")
prompt_template_name.format(cuisine='Indian')

'I want to open a restaurant for Indian food. Suggest a fency name for this.'

In [12]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template_name)
chain.run("Indian")

'\n\n"Spice Palace" '

In [18]:
llm = OpenAI(temperature=0.6)

prompt_template_name = PromptTemplate(
    input_variables=['cuisine'],
    template= "I want to open a restaurant for {cuisine} food. Suggest a fency name for this.")
name_chain = LLMChain(llm=llm, prompt=prompt_template_name)

prompt_template_menu = PromptTemplate(
    input_variables=['restaurant_name'],
    template= "Suggest some menu items for {restaurant_name}. Return it as a comma separated list.")
menu_chain = LLMChain(llm=llm, prompt=prompt_template_menu)

In [20]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains = [name_chain,menu_chain])
food_menu = chain.run('Indian')
print(food_menu)



1. Spicy Chicken Tikka Masala
2. Aloo Gobi (spiced cauliflower and potatoes)
3. Lamb Vindaloo
4. Tandoori Shrimp
5. Vegetable Samosas
6. Chana Masala (spiced chickpeas)
7. Palak Paneer (spinach and cheese curry)
8. Butter Chicken
9. Tandoori Chicken
10. Garlic Naan bread
11. Vegetable Biryani
12. Mango Lassi (spiced yogurt drink)
13. Chicken Korma
14. Dal Makhani (spiced lentil curry)
15. Gajar Halwa (spiced carrot dessert)


In [21]:
llm = OpenAI(temperature=0.6)

prompt_template_name = PromptTemplate(
    input_variables=['cuisine'],
    template= "I want to open a restaurant for {cuisine} food. Suggest a fency name for this.")
name_chain = LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaurant_name")

prompt_template_menu = PromptTemplate(
    input_variables=['restaurant_name'],
    template= "Suggest some menu items for {restaurant_name}. Return it as a comma separated list.")
menu_chain = LLMChain(llm=llm, prompt=prompt_template_menu, output_key="menu_items")

In [23]:
from langchain.chains import SequentialChain
chain = SequentialChain(
    chains=[name_chain,menu_chain],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name','menu_items']
)
chain({'cuisine':'Indian'})

{'cuisine': 'Indian',
 'restaurant_name': '\n\n"Spice Palace"',
 'menu_items': '\n\n1. Spicy Chicken Curry\n2. Vegetable Biryani\n3. Lamb Vindaloo\n4. Tandoori Chicken\n5. Aloo Gobi\n6. Garlic Naan\n7. Samosas\n8. Mango Lassi\n9. Chicken Tikka Masala\n10. Palak Paneer\n11. Chana Masala\n12. Vegetable Korma\n13. Butter Chicken\n14. Dal Makhani\n15. Gulab Jamun.'}

## AGENTS

In [31]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain_community.llms import OpenAI

tools = load_tools(["wikipedia","llm-math"],llm=llm)
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, #Does a reasoning  thought before action
    verbose=True
)
# agent.invoke("When was Elon Musk born? What will be his age in 2023?")
agent.invoke("What is OpenAI? What is GPT-4o and which date was it released?")



> Entering new AgentExecutor chain...
 Since the question is about two different things, I should probably use Wikipedia to answer the first question and Calculator for the second question.
Action: wikipedia
Action Input: OpenAI
Observation: Page: OpenAI
Summary: OpenAI is an American artificial intelligence (AI) research organization founded in December 2015, researching artificial intelligence with the goal of developing "safe and beneficial" artificial general intelligence, which it defines as "highly autonomous systems that outperform humans at most economically valuable work".
As one of the leading organizations of the AI boom, it has developed several large language models, advanced image generation models, and previously, released open-source models. Its release of ChatGPT has been credited with starting the AI boom.
The organization consists of the non-profit OpenAI, Inc. registered in Delaware and its for-profit subsidiary OpenAI Global, LLC. It was founded by Ilya Sutskever

{'input': 'What is OpenAI? What is GPT-4o and which date was it released?',
 'output': 'OpenAI is an artificial intelligence research organization that has developed various language and image models, including ChatGPT and Codex. It was founded in 2015 and has received significant investments from Microsoft. Sora is an upcoming text-to-video model being developed by OpenAI. GPT-4o does not seem to be a known model or release from OpenAI.'}

In [34]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain_community.llms import OpenAI

tools = load_tools(["serpapi","llm-math"],llm=llm)
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, #Does a reasoning  thought before action
    verbose=True
)
# agent.invoke("When was Elon Musk born? What will be his age in 2023?")
agent.invoke("What is OpenAI? What is GPT-4o and which date was it released?")



> Entering new AgentExecutor chain...
 I should use Search to find information about OpenAI and GPT-4o.
Action: Search
Action Input: OpenAI GPT-4o release date
Observation: ['GPT-4o has a 128K context window and has a knowledge cut-off date of October 2023. Finally, in terms of modalities: GPT-4o in the API supports ...', 'Prior to this update, GPT-4, which came out in March 2023, was available via the ChatGPT Plus subscription for $20 a month. It uses 1 trillion ...', "Discover OpenAI's GPT-4o and learn about its launch date, unique features, capabilities, cost, and practical use cases.", 'Following the pivotal date of May 13th, a series of subsequent releases for the timeline feature were meticulously scheduled, each unveiling ...', "It's a departure from the company's previous rollout of GPT-4, which was released in March 2023 for those who pay OpenAI's $20-per-month ...", "It was launched on March 14, 2023, and made publicly available via the paid chatbot product ChatGPT Plus, vi

{'input': 'What is OpenAI? What is GPT-4o and which date was it released?',
 'output': 'GPT-4o was released on May 13, 2024.'}

## MEMORY

In [38]:
dir(chain.memory)

['__bool__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [37]:
type(chain.memory)

NoneType

In [40]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

chain = LLMChain(llm=llm, prompt=prompt_template_name, memory=memory)
name = chain.run("Mexican")
print(name)



"El Sabroso Cantina" 


In [41]:
name = chain.run("Indian")
print(name)



"Maharaja's Table"


In [43]:
print(chain.memory.buffer)

Human: Mexican
AI: 

"El Sabroso Cantina" 
Human: Indian
AI: 

"Maharaja's Table"


In [46]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm=OpenAI(temperature=0.7))
print(convo.prompt)
print("############")
print(convo.prompt.template)

input_variables=['history', 'input'] template='The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\n\nCurrent conversation:\n{history}\nHuman: {input}\nAI:'
############
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [47]:
convo.run("who won the first cricket world cup?")

" The first Cricket World Cup was held in 1975 and was won by the West Indies. The West Indies defeated Australia by 17 runs in the final match held at Lord's Cricket Ground in London. The tournament was a round-robin format with eight teams competing. The West Indies had a dominant performance, winning all their matches except for one tie against Sri Lanka. The captain of the West Indies team at the time was Clive Lloyd, and the team was known for its strong batting lineup and fast bowlers."

In [48]:
convo.run("what is 5 + 5?")

'  5 + 5 is 10.'

In [49]:
convo.run("Who was the captain of the winning team?")

' The captain of the winning team was Clive Lloyd.'

In [52]:
print(convo.memory.buffer)

Human: who won the first cricket world cup?
AI:  The first Cricket World Cup was held in 1975 and was won by the West Indies. The West Indies defeated Australia by 17 runs in the final match held at Lord's Cricket Ground in London. The tournament was a round-robin format with eight teams competing. The West Indies had a dominant performance, winning all their matches except for one tie against Sri Lanka. The captain of the West Indies team at the time was Clive Lloyd, and the team was known for its strong batting lineup and fast bowlers.
Human: what is 5 + 5?
AI:   5 + 5 is 10.
Human: Who was the captain of the winning team?
AI:  The captain of the winning team was Clive Lloyd.


In [57]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)
convo = ConversationChain(
    llm=OpenAI(temperature=0.6),
    memory=memory
)
convo.run("who won the first cricket world cup?")

' The first Cricket World Cup was held in 1975 and was won by the West Indies. They defeated Australia in the final by 17 runs. The tournament was held in England and was a 60-over per side competition. The West Indies team was led by Clive Lloyd and had players like Viv Richards, Gordon Greenidge, and Michael Holding. It was a historic moment for the West Indies, as they became the first team to win the Cricket World Cup.'

In [59]:
convo.run("what is 5 + 5?")

' 5 + 5 is equal to 10.'

In [60]:
convo.run("who was the captain of the winning team?")

" I don't have enough information to answer that question. Can you provide more context?"